In [ ]:
import numpy as np
import pandas as pd

def get_od_path(edge,node,station2node_dict,gdtollnode):
    """
    获取OD路径和长度信息的函数。

    输入参数：
    - edge: 包含路段信息的DataFrame，包括 'u'（起点节点ID）、'v'（终点节点ID）、'length'（路段长度）、'edge_id'（路段ID）等列。
    - node: 包含节点信息的DataFrame，至少包括 'id'（节点ID）列。
    - station2node_dict: 字典，将站点ID映射到节点ID的关系。
    - gdtollnode: 包含站点信息的DataFrame，至少包括 'id' 列。

    输出结果：
    - od_dis_table: 包含每个OD对应的路段信息的DataFrame，包括 'station_id_x'（起点站点ID）、'station_id_y'（终点站点ID）、'edge_id'（路段ID）、'cumsumlength'（经过路段的累计长度）等列。
    - od_length: 包含每个OD对应的路径总长度的DataFrame，包括 'station_id_x'（起点站点ID）、'station_id_y'（终点站点ID）、'length'（路径总长度）等列。

    注意事项：
    - 函数中使用了外部库NetworkX来计算最短路径，请确保已经正确安装并导入NetworkX库。
    - 输入的DataFrame需要按照函数要求包含相应的列名和数据。
    - 输出结果为包含路径和长度信息的DataFrame，可以根据需要进一步分析和处理。

    """
    # 获取最短路径的函数（已优化，使用缓存）
    path_cache = {}  # 缓存已计算的最短路径
    

    def get_shortest_paths(start_station, end_station):
        if (start_station, end_station) not in path_cache:
            start_node = station2node_dict[start_station]
            end_node = station2node_dict[end_station]
            # 使用 shortest_simple_paths 获取最短的三条路径
            paths_generator = nx.shortest_simple_paths(G, source=start_node, target=end_node, weight='weight')
            shortest_paths = list(islice(paths_generator, 3))  # 获取前三条路径
            path_cache[(start_station, end_station)] = [list(map(int, path)) for path in shortest_paths]
        return path_cache[(start_station, end_station)]

    
    def get_path_dis_table(row):
        paths = row['paths']  # 注意这里是多条路径
        all_paths_details = []
        for path_id, path in enumerate(paths):
            path_details = [{'u': int(u), 'v': int(v), 'id': i,
                            'station_id_x': row['station_id_x'], 'station_id_y': row['station_id_y'],
                            'path_id': path_id}  # 添加路径标识符
                            for i, (u, v) in enumerate(zip(path[:-1], path[1:]))]
            all_paths_details.extend(path_details)
        return all_paths_details

    G_edges = edge[['u','v','length']].values
    G_nodes = list(node['id'])

    #先创建一个有向图
    G = nx.DiGraph()
    #添加节点
    G.add_nodes_from(G_nodes) 
    #添加边
    G.add_weighted_edges_from(G_edges)

    # 创建OD表
    o = gdtollnode[['id']]
    o.columns = ['station_id']
    o['flag'] = 1
    d = o.copy()
    od = pd.merge(o, d, on='flag')[['station_id_x', 'station_id_y']]
    od = od[od['station_id_x'] != od['station_id_y']]


    print('获取OD的出行路径')
    od['paths'] = od.apply(lambda r: get_shortest_paths(r['station_id_x'], r['station_id_y']), axis=1)

    # 调整整理OD的出行路径的代码
    print('整理OD的出行路径')
    od_details = od.apply(lambda r: get_path_dis_table(r), axis=1)
    # 对OD编号
    od['odid'] = range(len(od))

    # 使用并行处理来提高效率
    print('整理OD的出行路径')
    od_details = od.apply(lambda row: get_path_dis_table(row), axis=1).explode().tolist()
    od_details_flat = [item for sublist in od_details for item in sublist]  # 展开列表
    od1_tmp = pd.DataFrame(od_details_flat)

    # 合并得到的路径信息，避免多层循环
    od1_tmp = pd.DataFrame([item for sublist in od_details for item in sublist])

    # 合并边的信息
    od1_tmp = pd.merge(od1_tmp, edge[['u', 'v', 'length', 'edge_id']], on=['u', 'v'])

    # 转换为整型并排序
    od1_tmp['length'] = od1_tmp['length'].astype(int)
    od1_tmp = od1_tmp.sort_values(by=['station_id_x', 'station_id_y', 'id'])

    # 计算累计长度
    od1_tmp['cumsumlength'] = od1_tmp.groupby(['station_id_x', 'station_id_y', 'path_id'])['length'].cumsum()

    od_dis_table = od1_tmp[['station_id_x', 'station_id_y', 'edge_id', 'cumsumlength', 'path_id']]
    # od_dis_table存储了每个OD对应的路段信息
    # 其中，station_id_x、station_id_y为OD的起点、终点
    # edge_id为经过路段的id，cumsumlength为经过路段的累计长度
    # 例如，station_id_x=0，station_id_y=1，edge_id=47784，cumsumlength=68.596574，表示OD为0-1的出行路径，需要经过id为47784的路段，路径走完这一路段时，所经过的长度为68.596574米
    od_length = od1_tmp.groupby(['station_id_x', 'station_id_y', 'path_id'])['cumsumlength'].max().reset_index()
    od_length.rename(columns={'cumsumlength': 'length'}, inplace=True)

    return od_dis_table,od_length



def add_path_selection_probabilities(od_dis_table, od_length, beta=1.0):
    """
    计算每条路径的选择概率，并将这些概率加入到od_dis_table中。
    
    参数:
    - od_dis_table: 包含每个OD对的路径详细信息的DataFrame。
    - od_length: 包含每个OD对的路径长度信息的DataFrame。
    - beta: 模型参数，用于计算路径选择概率。默认值为1.0。
    
    返回:
    - 修改后的od_dis_table，现在包含每条路径的选择概率。
    """
    # 计算每条路径的选择概率
    od_length['exp_neg_beta_length'] = np.exp(-beta * od_length['length'])
    od_length['sum_exp_neg_beta_length'] = od_length.groupby(['station_id_x', 'station_id_y'])['exp_neg_beta_length'].transform('sum')
    od_length['probability'] = od_length['exp_neg_beta_length'] / od_length['sum_exp_neg_beta_length']
    
    # 准备合并所需的概率信息
    probabilities = od_length[['station_id_x', 'station_id_y', 'path_id', 'probability']]
    
    # 将选择概率合并到od_dis_table中
    od_dis_table_updated = pd.merge(
        od_dis_table,
        probabilities,
        on=['station_id_x', 'station_id_y', 'path_id'],
        how='left'
    )
    
    return od_dis_table_updated   

def od_merge_table_reconstruct(od_dis_table):
    """
    重新构造OD合并表格的函数，增加了对每个OD对的路径ID和对应选择概率的处理。

    输入参数：
    - od_dis_table: 包含每个OD对应的路段信息的DataFrame，需要包含 'station_id_x', 'station_id_y', 'edge_id', 'cumsumlength', 'path_id', 和 'probability' 列。

    输出结果：
    - od_merge_table: 重新构造的OD合并表格的DataFrame，包括 'station_id_x', 'station_id_y', 'edge_id', 'cumsumlength', 'cumsumlength2', 'path_ids', 和 'probabilities' 列。
    """
    # 合并列表，并添加对 path_id 和 probability 的处理
    od_merge_table = od_dis_table.groupby(['station_id_x', 'station_id_y']).agg({
        'edge_id': list,
        'cumsumlength': list,
        'path_id': lambda x: list(set(x)),  # 使用 set 来去重，然后转回 list
        'probability': list
    }).reset_index()
    
    # 计算累计长度差值
    def calculate_differences(lst):
        differences = [lst[i] - lst[i - 1] for i in range(1, len(lst))]
        return [lst[0]] + differences
    
    od_merge_table['cumsumlength2'] = od_merge_table['cumsumlength']
    od_merge_table['cumsumlength'] = od_merge_table['cumsumlength'].parallel_apply(calculate_differences)
    
    # 重命名列以清晰表示它们的内容
    od_merge_table.rename(columns={'path_id': 'path_ids', 'probability': 'probabilities'}, inplace=True)

    return od_merge_table

import numpy as np
import pandas as pd

def get_agent_path_table(agent_data, new_od_dis):
    """
    根据代理数据和新的OD距离信息创建代理路径表格，并基于路径概率进行抽样。

    参数:
    agent_data (DataFrame): 包含代理信息的数据框。
    new_od_dis (DataFrame): 包含OD距离和路径选择概率信息的数据框。

    返回值:
    DataFrame: 包含代理路径信息和抽样确定的路径累计长度的数据框。
    """
    # 保留所需的代理信息列
    agent_data = agent_data[['trip_id', 'ids', 'O', 'D', 'date', 'start_step']].copy()

    # 准备数据框以进行合并
    merged_data = pd.DataFrame()

    for index, agent in agent_data.iterrows():
        # 根据代理的起点和终点匹配OD信息
        od_paths = new_od_dis[(new_od_dis['station_id_x'] == agent['O']) & (new_od_dis['station_id_y'] == agent['D'])]

        if not od_paths.empty:
            # 基于路径概率进行抽样选择路径
            path_ids = od_paths['path_ids'].iloc[0]
            probabilities = od_paths['probabilities'].iloc[0]
            chosen_path_id = np.random.choice(path_ids, p=probabilities)
            
            # 选择对应的路径和累计长度
            chosen_path_info = od_paths[od_paths['path_ids'].apply(lambda x: chosen_path_id in x)]
            chosen_path = chosen_path_info['edge_id'].iloc[0]
            chosen_cumsumlength = chosen_path_info['cumsumlength2'].iloc[0]
            
            # 添加选择的路径信息到代理数据
            agent_data.loc[index, 'path'] = chosen_path
            agent_data.loc[index, 'cumsumlength'] = chosen_cumsumlength

    return agent_data